In [1]:
# Python 2
import cv2
import numpy as np
import pandas as pd
import random
from __future__ import print_function
from os import listdir
from os.path import isfile, join
from pandas.io.pickle import read_pickle
from scipy.spatial.distance import cosine as cos_dist
# from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
def init_df(img_dir):
    df = pd.DataFrame(filter(lambda x: ".JPG" in x, listdir(img_dir)), columns=["name"])
    df["class"] = np.repeat(np.linspace(0, 49, num=50), 4)
    df["class"] = df["class"].astype("category")
    key_points, kp_descriptors = list(), list()
    df_name = list(df["name"])
    sift = cv2.SIFT(nfeatures=20)
    for i in range(0, 200):
        progress = int(i / 200.0 * 1000) / 10.0
        print("{} {}%".format(df_name[i], progress), end="\r")
        img = cv2.imread(join(img_dir, df_name[i]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(img, None)
        key_points.append(map(lambda x: {"pt": x.pt, "angle": x.angle, "size": x.size,
                                         "octave": x.octave, "response": x.response}, kp))
        kp_descriptors.append(des.astype(np.uint16))
    df["sift_key_points"] = key_points
    df["sift_kp_descriptors"] = kp_descriptors
    return df

In [3]:
df_cache = "df.pickle"
if isfile(df_cache):
    df = read_pickle(df_cache)
else:
    df = init_df("data")
    df.to_pickle(df_cache)
print(df.shape)
print(df.dtypes)
print(df.head())

(200, 4)
name                     object
class                  category
sift_key_points          object
sift_kp_descriptors      object
dtype: object
           name class                                    sift_key_points  \
0  image001.JPG     0  [{u'response': 0.0780883580446, u'angle': 296....   
1  image002.JPG     0  [{u'response': 0.0773346424103, u'angle': 84.5...   
2  image003.JPG     0  [{u'response': 0.086534768343, u'angle': 91.15...   
3  image004.JPG     0  [{u'response': 0.059586878866, u'angle': 279.4...   
4  image006.JPG     1  [{u'response': 0.0622775293887, u'angle': 269....   

                                 sift_kp_descriptors  
0  [[3, 121, 65, 3, 7, 3, 5, 3, 4, 41, 54, 39, 12...  
1  [[7, 19, 8, 1, 1, 0, 16, 16, 6, 16, 20, 11, 1,...  
2  [[138, 2, 0, 0, 55, 7, 0, 3, 53, 10, 0, 0, 142...  
3  [[10, 42, 7, 2, 84, 81, 6, 0, 74, 109, 3, 2, 8...  
4  [[100, 7, 0, 0, 22, 35, 44, 53, 61, 5, 0, 0, 1...  


In [4]:
def new_split():
    split = ["train"] * 200
    for i in range(0, 50):
        split[i * 4 + random.randint(0, 3)] = "test"
    return split
for i in range(0, 10):
    split_name = "split_" + str(i)
    df[split_name] = new_split()
    df[split_name] = df[split_name].astype("category")
print(df.shape)
print(df.head())

(200, 14)
           name class                                    sift_key_points  \
0  image001.JPG     0  [{u'response': 0.0780883580446, u'angle': 296....   
1  image002.JPG     0  [{u'response': 0.0773346424103, u'angle': 84.5...   
2  image003.JPG     0  [{u'response': 0.086534768343, u'angle': 91.15...   
3  image004.JPG     0  [{u'response': 0.059586878866, u'angle': 279.4...   
4  image006.JPG     1  [{u'response': 0.0622775293887, u'angle': 269....   

                                 sift_kp_descriptors split_0 split_1 split_2  \
0  [[3, 121, 65, 3, 7, 3, 5, 3, 4, 41, 54, 39, 12...   train   train   train   
1  [[7, 19, 8, 1, 1, 0, 16, 16, 6, 16, 20, 11, 1,...   train   train   train   
2  [[138, 2, 0, 0, 55, 7, 0, 3, 53, 10, 0, 0, 142...    test    test   train   
3  [[10, 42, 7, 2, 84, 81, 6, 0, 74, 109, 3, 2, 8...   train   train    test   
4  [[100, 7, 0, 0, 22, 35, 44, 53, 61, 5, 0, 0, 1...   train    test   train   

  split_3 split_4 split_5 split_6 split_7 split_8 sp

In [5]:
def get_training_set(split_id):
    df_train = df[df["split_" + str(split_id)] == "train"]
    df_train_sift_des = list(df_train["sift_kp_descriptors"])
    df_train_class = list(df_train["class"])
    train_x, train_y = list(), list()
    for i in range(0, 150):
        for d in df_train_sift_des[i]:
            train_x.append(d)
            train_y.append(df_train_class[i])
    return train_x, train_y

In [6]:
def test(classifier, split_id):
    df_test = df[df["split_" + str(split_id)] == "test"]
    df_test_sift_des = list(df_test["sift_kp_descriptors"])
    df_test_name = list(df_test["name"])
    predict = np.zeros(50, dtype=np.uint8)
    for i in range(0, 50):
        progress = int(i / 50.0 * 1000) / 10.0
        print("split_{} {} {}%".format(split_id, df_test_name[i], progress), end="\r")
        vote = np.zeros(50, dtype=np.uint16)
        for d in df_test_sift_des[i]:
            vote[classifier.predict(d)[0]] += 1
        predict[i] = vote.argmax()
    return predict

In [8]:
classifier = KNeighborsClassifier(n_neighbors=5, weights="distance", metric=cos_dist)
# scipy.spatial.distance.cosine is much faster than sklearn.metrics.pairwise.cosine_distances
accuracy = np.zeros(10, dtype=np.uint8)
for i in range(0, 10):
    train_x, train_y = get_training_set(i)
    predict = test(classifier.fit(train_x, train_y), i)
    accuracy[i] = sum(np.linspace(0, 49, num=50) == predict)
print(accuracy)
print("mean={}, sigma={}".format(np.mean(accuracy), np.std(accuracy)))

[42 41 39 40 38 39 42 42 42 38]
mean=40.3, sigma=1.61554944214
